#Homework2

In [ ]:
this homework basic on teacher's work-'05 CDL Analysis'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import pyproj
import rasterio
import rasterstats
import pandas as pd
import json
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib.colors as pltc
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats

In [ ]:
data_dir ="C:\Users\27354\Desktop\California\Central_Valley_CDL_2017_clip.tif"
data_dir ="C:\Users\27354\Desktop\California\Central_Valley_CDL_2018_clip.tif"

In [ ]:

cdl_17 = rasterio.open(data_dir + "Central_Valley_CDL_2017_clip.tif")
cdl_18 = rasterio.open(data_dir + "Central_Valley_CDL_2018_clip.tif")

In [ ]:
country = geopandas.read_file(data_dir+"us_states.json")

In [ ]:
cdls = {17:cdl_17,18:cdl_18}

In [ ]:
cdl_18.crs == country.crs

In [ ]:
# Reproject
area = country.to_crs(cdl_17.crs)

In [ ]:
area = country[(country.NAME!="Central Valley")]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
rasterio.plot.show(cdl_17, ax=ax)
area.plot(ax=ax, facecolor='none', edgecolor='black')

In [ ]:
def getFeatures(gdf):
    import json
    return [json.loads(gdf.to_json())['features'][0]['Central Valley']]

In [ ]:
L_coords = getFeatures(country[(country["NAME"] == "Central Valley")])

coords = \
{"IL": IL_coords}

img_out, img_transform = mask(cdl_17,IL_coords) 

In [ ]:
plt.imshow(img_out[0,:,:])

In [ ]:
plt.plot((img_out[0,:,:]!=0).sum(axis=0))

In [ ]:
plt.imshow(img_out[0,:,:15000])

In [ ]:
for year,cdl in cdls.items():
    for k,v in coords.items():
        # Transform with the mask
        img_out, img_transform = mask(cdl,v) 
        
        # Cut out the right side of the image.
        img_out = img_out[:,:,:15000]
        
        # Metadata
        out_meta = cdl.meta.copy()
        out_meta['height'] = img_out.shape[1]
        out_meta['width'] = img_out.shape[2]
        out_meta['transform'] = img_transform
        
        # This field will tell you which pixel values mean "no data" and thus are transparent.
        out_meta['nodata'] = 0
        
        with rasterio.open("./raster_"+k+"_"+str(year)+".tif", "w", **out_meta) as dest:
            dest.write(img_out)

In [ ]:
IL_17 = rasterio.open("./raster_IL_17.tif")
IL_18 = rasterio.open("./raster_IL_18.tif")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
rasterio.plot.show(IL_17, ax=ax)
country[country['NAME']=="Illinois"].plot(ax=ax, facecolor='none', edgecolor='black')

In [ ]:
with open("CDL_dict.json") as f:
    pixel_names = json.load(f)[0]

In [ ]:
rasters = {}
for year in cdls.keys():
    rasters[year] = {}
    for state in coords.keys():
        rasters[year][state] = rasterio.open("./raster_"+state+"_"+str(year)+".tif")

In [ ]:
r17_array = rasters[17]['IL'].read(1)
pixels = pd.Series(r17_array.flatten())
pixels = pixels[pixels!=0]

pixel_counts = pixels.value_counts()
pixel_counts.head(10)

In [ ]:
pixel_ixs = pd.Series(pixel_counts.index).astype(str)
pixel_counts.index = pixel_ixs.replace('Crops (including Fallowed Land)')
pixel_counts.index = pixel_ixs.replace('forest')
pixel_counts.index = pixel_ixs.replace('Grassland/Pasture')
pixel_counts.index = pixel_ixs.replace("Shrubland")
pixel_counts.index = pixel_ixs.replace('Developed Land')
pixel_counts.index = pixel_ixs.replace('Other')
pixel_counts.head(10)

In [ ]:
(pixel_counts/pixel_counts.sum()*100).head(10)

In [ ]:
pixel_counts = pixels.value_counts(normalize=True)*100

pixel_ixs = pd.Series(pixel_counts.index).astype(str)

pixel_counts.index = pixel_ixs.replace(pixel_names)

pixel_counts.head(10)

In [ ]:
pixel_counts.plot(kind="pie")

In [ ]:
condensed_counts = pixel_counts.iloc[:4]
condensed_counts.loc["Other"] = pixel_counts.iloc[4:].sum()
condensed_counts.plot(kind="pie")

In [ ]:
def recode_raster_binary(raster,array,img_transform,name):
    out_meta = raster.meta.copy()
    array = array[np.newaxis,:,:]
    out_meta['height'] = array.shape[1]
    out_meta['width'] = array.shape[2]
    out_meta['transform'] = img_transform
    
    with rasterio.open(name, "w", **out_meta) as dest:
        dest.write(array.astype(rasterio.uint8))

In [ ]:
area_labels = {1:"cornland",5:"fallowland"}

In [ ]:
cropland_rasters = {}

# For each state
for state in coords.keys():
    cropland_rasters[state] = {}
    # For each CDL year:
    for year in cdls.keys():
        cropland_rasters[state][year] = {}
        # For each crop:
        for val,crop in area_labels.items():
            # Read in the band and make a binary array:
            array = (rasters[year][state].read(1)==val).astype(np.int8)
            
            # Write it out using our function
            recode_raster_binary(cdl,array,img_transform,"./raster_"+state+"_"+str(year)+"_"+crop+".tif")

            # Store it in this dictionary:
            cropland_rasters[state][year][crop] = rasterio.open("./raster_"+state+"_"+str(year)+"_"+crop+".tif")

In [ ]:
# Make some colormaps first:

cmap_cropland = matplotlib.colors.ListedColormap(['white',"C0"])
cmap_fallowland = matplotlib.colors.ListedColormap(['white',"darkgreen"])

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(10, 15))

# Cornland 2017
rasterio.plot.show(cropland_rasters["IL"][17]["cornland"], ax=ax[0,0],cmap=cmap_cropland)
ax[0,0].set_title("Corn Area, 2017")
country[country['NAME']=="valley"].plot(ax=ax[0,0], facecolor='none', edgecolor='black')

# fallowland 2017
rasterio.plot.show(cropland_rasters["IL"][17]["fallowland"], ax=ax[0,1],cmap=cmap_fallowland)
ax[0,1].set_title("fallowland, 2017")
country[country['NAME']=="valley"].plot(ax=ax[0,1], facecolor='none', edgecolor='black')

# Cornland 2017
rasterio.plot.show(cropland_rasters["IL"][18]["cornland"], ax=ax[1,0],cmap=cmap_cropland)
ax[1,0].set_title("Corn Area, 2018")
country[country['NAME']=="valley"].plot(ax=ax[1,0], facecolor='none', edgecolor='black')

# fallowland 2018
rasterio.plot.show(cropland_rasters["IL"][18]["fallowland"], ax=ax[1,1],cmap=cmap_fallowland)
ax[1,1].set_title("fallowland, 2018")
country[country['NAME']=="valley"].plot(ax=ax[1,1], facecolor='none', edgecolor='black')

In [ ]:

IL_18 = np.concatenate([rasterio.open("./raster_IL_18_cornland.tif").read(1)[np.newaxis,:,:], 
                        rasterio.open("./raster_IL_18_fallowland.tif").read(1)[np.newaxis,:,:]],axis=0) 

IL_17 = np.concatenate([rasterio.open("./raster_IL_17_cornland.tif").read(1)[np.newaxis,:,:],
                        rasterio.open("./raster_IL_17_fallowland.tif").read(1)[np.newaxis,:,:]],axis=0)

In [ ]:
# Assign a one to anything monocropped
cornland_to_cornland = ((IL_18[0,:,:]==1)&(IL_17[0,:,:]==1))*1
fallowland_to_fallowland = ((IL_18[1,:,:]==1)&(IL_17[1,:,:]==1))*1

# Assign a two to anything that rotated
fallowland_to_cornland = ((IL_18[0,:,:]==1)&(IL_17[1,:,:]==1))*2
cornland_to_fallowland = ((IL_18[1,:,:]==1)&(IL_17[0,:,:]==1))*2

In [ ]:
Rotated = cornland_to_cornland + fallowland_to_fallowland + fallowland_to_cornland + cornland_to_fallowland

In [ ]:
# First put it in a raster format:
recode_raster_binary(cdl_17,Rotated,img_transform,"./raster_IL_rotation.tif")

Rotated_Raster = rasterio.open("./raster_IL_rotation.tif")

In [ ]:
labels = {1:"Non-Rotated",
          2:"Rotated"}

cmap = matplotlib.colors.ListedColormap(['white', "C3","C0"])

patches =[mpatches.Patch(color=cmap(i),label=labels[i]) for i in labels.keys()]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
rasterio.plot.show(Rotated_Raster,cmap=cmap,ax=ax)
country[country['NAME']=="valley"].plot(ax=ax, facecolor='none', edgecolor='black')
plt.legend(handles=patches,fontsize=16)
plt.axis('off')

In [ ]:
# As before:
cornland_to_cornland = ((IL_18[0,:,:]==1)&(IL_17[0,:,:]==1))*1
fallowland_to_fallowland = ((IL_18[1,:,:]==1)&(IL_17[1,:,:]==1))*2
fallowland_to_cornland = ((IL_18[0,:,:]==1)&(IL_17[1,:,:]==1))*3
cornland_to_fallowland = ((IL_18[1,:,:]==1)&(IL_17[0,:,:]==1))*4

In [ ]:
Rotated_Cats = cornland_to_cornland + fallowland_to_fallowland + fallowland_to_cornland + cornland_to_fallowland

In [ ]:
for i in range(Rotation.shape[0]):
    for j in range(Rotation.shape[1]):
        if (IL_18[0,i,j]==1)&(IL_17[0,i,j]==1):
            Rotation[i,j] = 1
        elif (IL_18[1,i,j]==1)&(IL_17[1,i,j]==1):
            Rotation[i,j] = 2
        elif (IL_18[0,i,j]==1)&(IL_17[1,i,j]==1):
            Rotation[i,j] = 3
        elif (IL_18[1,i,j]==1)&(IL_17[0,i,j]==1):
            Rotation[i,j] = 4

In [ ]:
recode_raster_binary(cdl_17,Rotated_Cats,"./raster_IL_rotation_cats.tif")

Rotated_Cat_Raster = rasterio.open("./raster_IL_rotation_cats.tif")

In [ ]:
cmap = matplotlib.colors.ListedColormap(['white',"C1","C3", 'C2', 'C0'])

labels = {1:"Cornland to Cornland",
          2:"Fallowland to Fallowland",
          3:"Fallowland to Cornland",
          4:"Cornland to Fallowland"}

patches =[mpatches.Patch(color=cmap(i),label=labels[i]) for i in labels.keys()]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
rasterio.plot.show(Rotated_Cat_Raster,cmap=cmap,ax=ax)
country[country['NAME']=="Valley"].plot(ax=ax, facecolor='none', edgecolor='black')
plt.legend(handles=patches,fontsize=12,loc=(0,.8))
plt.axis('off')

In [ ]:
tated_Series = pd.Series(Rotated_Cats.flatten())

top_transitions = Rotated_Series[Rotated_Series!=0].value_counts(normalize=True)*100

labels = {"1":"Cornland to Cornland",
          "2":"Fallowland to Fallowland",
          "3":"Fallowland to Fallowland",
          "4":"Cornland to Fallowland"}

top_transitions.index = pd.Series(top_transitions.index).astype(str).replace(labels)

top_transitions

What data sources you would need in addition to the data you have now.
To answer this question，Basically, I need the rainfall map of the Valley in California in 2017. More accurately, I need the amount of rainfall and the contour map of the Valley.

How you would manipulate the data.
First,I will read these data,because I want to know the rainfall map how to change the phenomenon that fallow land to crop land.Then,I will compare rainfall map and CDL in 2017, to ensure that rainfall truely infact the land use.Third, I will use the data to refine the changes in the specific use of land. Finally, I will use the data to calculate the impact ratio of rainfall on land transformation, and make appropriate charts based on these data.

What statistics you would calculate or model you would run on this data
SWAT model.

How you might present your findings.
I will use PPT to show my findings, more I also wants to has a speech to show how to rational use of land resources.

what factors would make it difficult to accurately answer this question? List three and explain why they might make answering the question difficult. 
Databases is too few. And too hard to find data, especially different kinds of data.
May we choose the model is not accurate.
Accurate calculation requires practical investigation
So, these will make answering the question difficult.